In [1]:
import random
import logging
from IPython.display import display, HTML
import os 

import numpy as np
import pandas as pd
import datasets
from datasets import load_dataset, load_metric, ClassLabel, Sequence
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

from sklearn.model_selection import train_test_split

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
def seed_everything(seed: int = 17):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
seed_everything(17)

In [3]:
model_checkpoint = "test-nli/checkpoint-1142"
batch_size = 32
task = "nli"
MODEL_P = "models/klue-bert-base-augmented.pth"
RANDOM_SEED = 17
num_labels = 7


In [4]:
models = []
toeknizers = []

In [5]:
checkpoints = ["test-nli/checkpoint-1142","test-nli/checkpoint-571","test-nli/checkpoint-11415","test-nli/checkpoint-571","test-nli/checkpoint-3426"]

In [6]:
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [7]:
# tokenizer

In [8]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_key, label_key, bert_tokenizer):
        
        self.sentences = [ bert_tokenizer(i,truncation=True,return_token_type_ids=False) for i in dataset[sent_key] ]
        
        if not label_key == None:
            self.mode = "train"
        else:
            self.mode = "test"
            
        if self.mode == "train":
            self.labels = [np.int64(i) for i in dataset[label_key]]
        else:
            self.labels = [np.int64(0) for i in dataset[sent_key]]

    def __getitem__(self, i):
        if self.mode == "train":
            self.sentences[i]["label"] = self.labels[i]
            return self.sentences[i]
#             return ( self.sentences[i] , self.labels[i] )
        else:
            return self.sentences[i]

    def __len__(self):
        return (len(self.labels))

In [9]:
# model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [10]:
dataset = pd.read_csv("data/train_data.csv",index_col=False)
test = pd.read_csv("data/test_data.csv",index_col=False)

In [11]:
metric = load_metric("glue", "qnli")

In [12]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
metric_name = "accuracy"

args = TrainingArguments(
    "test-nli",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [26]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5)
total_pred = np.zeros((9131,7),dtype=np.float64)
result = []

In [28]:
total_pred.shape

(9131, 7)

In [29]:
for step, (train_idx, test_idx) in enumerate(skf.split(dataset["title"],dataset["topic_idx"])):
    tokenizer = AutoTokenizer.from_pretrained(checkpoints[step], use_fast=True)
    model = AutoModelForSequenceClassification.from_pretrained(checkpoints[step], num_labels=num_labels)
    
    dataset_train = pd.DataFrame()
    dataset_val = pd.DataFrame()
    
    dataset_train["title"] = dataset["title"][train_idx]
    dataset_train["topic_idx"] = dataset["topic_idx"][train_idx]
    
    dataset_val["title"] = dataset["title"][train_idx]
    dataset_val["topic_idx"] = dataset["topic_idx"][train_idx]
    
    data_train = BERTDataset(dataset_train, "title", "topic_idx", tokenizer)
    data_val = BERTDataset(dataset_val, "title", "topic_idx", tokenizer)
    data_test = BERTDataset(test, "title", None, tokenizer)
    
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=data_train,
        eval_dataset=data_val,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
        )
    
    print(trainer.evaluate())
    pred = trainer.predict(data_test)
    pred = pred[0]
    print(pred)
    result.append(pred)
    total_pred += pred
    print(total_pred)

Didn't find file test-nli/checkpoint-1142\added_tokens.json. We won't load it.
loading file test-nli/checkpoint-1142\vocab.txt
loading file test-nli/checkpoint-1142\tokenizer.json
loading file None
loading file test-nli/checkpoint-1142\special_tokens_map.json
loading file test-nli/checkpoint-1142\tokenizer_config.json
loading configuration file test-nli/checkpoint-1142\config.json
Model config BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,

***** Running Prediction *****
  Num examples = 9131
  Batch size = 32


{'eval_loss': 0.3221838176250458, 'eval_accuracy': 0.8977630534183939, 'eval_runtime': 29.7215, 'eval_samples_per_second': 1228.84, 'eval_steps_per_second': 38.423}


Didn't find file test-nli/checkpoint-571\added_tokens.json. We won't load it.
loading file test-nli/checkpoint-571\vocab.txt
loading file test-nli/checkpoint-571\tokenizer.json
loading file None
loading file test-nli/checkpoint-571\special_tokens_map.json
loading file test-nli/checkpoint-571\tokenizer_config.json
loading configuration file test-nli/checkpoint-571\config.json
Model config BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "

[[ 1.8296647  -0.5886467   1.1916904  ... -1.8184546  -1.768165
  -1.8238086 ]
 [-0.56964153 -1.3956958  -0.34061196 ... -0.7699641  -0.9862497
  -1.0501513 ]
 [ 1.0363683  -0.03387248  2.7351336  ... -1.4208844  -1.6300248
  -0.59794676]
 ...
 [-1.5907278  -1.4128801   3.1635294  ... -0.7266785  -1.2603605
   0.03929582]
 [-0.81325316 -0.18853243  2.313022   ...  0.69500864 -0.9717218
  -0.73073107]
 [ 0.9129015  -0.27370366  2.0075054  ... -1.6280522  -2.0125625
   0.95499384]]
[[ 1.82966471 -0.58864671  1.19169044 ... -1.81845462 -1.76816499
  -1.82380855]
 [-0.56964153 -1.39569581 -0.34061196 ... -0.7699641  -0.98624969
  -1.05015135]
 [ 1.03636825 -0.03387248  2.73513365 ... -1.42088437 -1.63002479
  -0.59794676]
 ...
 [-1.59072781 -1.41288006  3.1635294  ... -0.72667849 -1.26036048
   0.03929582]
 [-0.81325316 -0.18853243  2.3130219  ...  0.69500864 -0.97172183
  -0.73073107]
 [ 0.91290152 -0.27370366  2.00750542 ... -1.62805223 -2.01256251
   0.95499384]]


...

Didn't find file test-nli/checkpoint-3426\added_tokens.json. We won't load it.
loading file test-nli/checkpoint-3426\vocab.txt
loading file test-nli/checkpoint-3426\tokenizer.json
loading file None
loading file test-nli/checkpoint-3426\special_tokens_map.json
loading file test-nli/checkpoint-3426\tokenizer_config.json
loading configuration file test-nli/checkpoint-3426\config.json
Model config BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,

[[ 2.2598155   0.13129948  0.8028287  ... -1.4706604  -1.1137404
  -1.8833427 ]
 [-0.42463085 -1.1062497   0.49260506 ... -0.37027526 -0.81788224
  -0.9205057 ]
 [ 1.4369446   1.148286    2.071815   ... -1.4183725  -0.8476488
  -0.48760745]
 ...
 [-0.9841947  -1.3189999   2.1849656  ... -0.7079726  -0.78494215
  -0.0338817 ]
 [ 0.15203664 -0.4153636   1.4171369  ...  0.4047152  -1.3455517
  -0.82939667]
 [ 1.4163818   0.7006415   2.1672654  ... -1.2832208  -0.865139
   0.03863312]]
[[ 8.41362953 -0.11592117  4.92171323 ... -6.76692808 -6.31423318
  -8.17509377]
 [-2.61428964 -4.88396358  0.65574212 ... -2.90446937 -3.99703312
  -3.86622721]
 [ 5.51869392  2.87795568 10.09465814 ... -6.30109143 -5.54401731
  -3.06145591]
 ...
 [-4.95175576 -4.87658209 11.04672694 ... -3.40885592 -4.60173488
  -1.61572865]
 [-0.26926646 -0.93423092  8.53740788 ...  2.55136865 -5.41687459
  -2.88773972]
 [ 4.79843378  0.84369287  8.77667165 ... -6.48084664 -5.48857546
   2.13948932]]


All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at test-nli/checkpoint-3426.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.
***** Running Evaluation *****
  Num examples = 36524
  Batch size = 32


***** Running Prediction *****
  Num examples = 9131
  Batch size = 32


{'eval_loss': 0.3067176938056946, 'eval_accuracy': 0.8982586792246194, 'eval_runtime': 30.8572, 'eval_samples_per_second': 1183.647, 'eval_steps_per_second': 37.009}
[[ 2.1311948  -0.43703285  1.3604176  ... -2.1165307  -2.2085876
  -2.1589136 ]
 [-0.91676015 -1.5316292  -0.24405459 ... -0.8290416  -1.1634315
  -1.0989068 ]
 [ 1.4303285  -0.07308714  3.0889943  ... -1.8894304  -2.0010583
  -0.57616544]
 ...
 [-1.5530137  -1.5394124   3.5624466  ... -0.9438938  -1.5631717
  -0.61235416]
 [-0.8754548  -0.17047238  2.6757092  ...  0.7074295  -1.3031926
  -0.9962959 ]
 [ 1.2663587  -0.464074    1.9526765  ... -1.9425809  -2.219511
   1.4895629 ]]
[[ 10.54482436  -0.55295402   6.28213084 ...  -8.88345873  -8.52282083
  -10.33400738]
 [ -3.53104979  -6.41559279   0.41168754 ...  -3.73351097  -5.16046464
   -4.96513397]
 [  6.94902241   2.80486854  13.1836524  ...  -8.19052184  -7.54507565
   -3.63762134]
 ...
 [ -6.50476944  -6.41599447  14.60917354 ...  -4.35274971  -6.16490662
   -2.228082

In [32]:
total = np.argmax(total_pred,axis=1)

In [34]:
len(total)

9131

In [35]:
submission = pd.read_csv('data/sample_submission.csv')
submission['topic_idx'] = total
submission.to_csv("results/klue-bert-base-kfold5-0804.csv",index=False)